# Libaries and prerequisites settings

In [1]:
import pandas as pd
import string
import re
from collections import defaultdict

In [2]:
# Kosaraju's algorithm to find strongly connected components in Python
# Code was modified from source : https://www.programiz.com/dsa/strongly-connected-components

class Graph:

    def __init__(self, vertex):
        self.V = vertex
        self.graph = defaultdict(list)

    def add_edge(self, s, d):
        self.graph[s].append(d)

    # dfs qui retourne le nombre de noeuds visités et donc fortement connexes
    def dfs(self, d, visited_vertex, reverse_dict):
        result_tab = [reverse_dict[d]];
        result_size = 1;
        visited_vertex[d] = True
        # print(reverse_dict[d], end=' ')
        for i in self.graph[d]:
            if not visited_vertex[i]:
                next_traversal = self.dfs(i, visited_vertex, reverse_mapping)
                result_tab,result_size = result_tab+next_traversal[0], result_size+next_traversal[1]
        return (result_tab,result_size)

    def fill_order(self, d, visited_vertex, stack):
        visited_vertex[d] = True
        for i in self.graph[d]:
            if not visited_vertex[i]:
                self.fill_order(i, visited_vertex, stack)
        stack = stack.append(d)

    # transposer la matrice
    def transpose(self):
        g = Graph(self.V)

        for i in self.graph:
            for j in self.graph[i]:
                g.add_edge(j, i)
        return g

    # trouver les satellites et le core
    def get_core_n_satellites(self, reverse_mapping, size=0):
        stack = []
        visited_vertex = [False] * (self.V)

        for i in range(self.V):
            if not visited_vertex[i]:
                self.fill_order(i, visited_vertex, stack)

        gr = self.transpose()

        visited_vertex = [False] * (self.V)
        res = [] # les satellites
        core_size = -1;
        core_ind = -1;
        core = [];
        
        while stack:
            i = stack.pop()
            if not visited_vertex[i]:
                current_strongly_connected = gr.dfs(i, visited_vertex, reverse_mapping)
                
                if(current_strongly_connected[1]) >= 2: # élimine les mots isolés / seuls
                    res.append(current_strongly_connected)
                    if(current_strongly_connected[1] > core_size):
                        core = current_strongly_connected[0]
                        core_size = current_strongly_connected[1]
                        core_ind = len(res)-1
                        
        del res[core_ind]
                
        return (core,res)

## Data preparation

In [3]:
df = pd.read_csv("dictionary.csv");

In [4]:
df.head()

,word,type,definition
0,Aam,n.,"A Dutch and German measure of liquids, varying..."
1,Aard-vark,n.,"An edentate mammal, of the genus Orycteropus, ..."
2,Aard-wolf,n.,"A carnivorous quadruped (Proteles Lalandii), o..."
3,Aaronic,a.,Alt. of Aaronical
4,Aaronical,a.,"Pertaining to Aaron, the first high priest of ..."


In [5]:
len(df)

54544

In [6]:
df.drop_duplicates(subset=['word'])
n = len(df) # number of words
del df['type'] # we don't need it
print(n)

54544


In [7]:
# lowercase characters
df['word'] = df['word'].str.lower()
df['definition'] = df['definition'].str.lower()


# take away special characters, except '-' for composed names and whitespaces
df['definition'].replace('[^-a-zA-Z\d\s:]','',regex=True, inplace=True)

In [8]:
df.head()

,word,definition
0,aam,a dutch and german measure of liquids varying ...
1,aard-vark,an edentate mammal of the genus orycteropus so...
2,aard-wolf,a carnivorous quadruped proteles lalandii of s...
3,aaronic,alt of aaronical
4,aaronical,pertaining to aaron the first high priest of t...


In [9]:
kernel = []
core = []
satellites = []

## Kernel extraction & graph generation

In [10]:
dictionnary = dict() # the definition of each word
id_mapping = dict() # graph node - word mapping
reverse_mapping = dict()
ind = 0
for index, row in df.iterrows():
    id_mapping[row['word']] = ind
    reverse_mapping[ind] = row['word']
    ind += 1
    if(row['word'] in dictionnary):
        dictionnary[row['word']][1] += row['definition'].split()
    else:
        dictionnary[row['word']] = [0, row['definition'].split()] # first value will serve for later ( kernel or not ? ), second is definition

In [11]:
non_existants = set()
count_kernel = 0

for key,value in dictionnary.items():
    for word in value[1]:
        if word in dictionnary:
            if(value[0]==0):
                count_kernel+=1
                kernel.append(word)
            value[0] = 1
        else:
            non_existants.add(word)
print(len(dictionnary))

34193


In [12]:
print("We see that there are , ", len(non_existants) , " non defined words in our dictionnary\nThis is not consisent with the assumption that a Dictionary (D) is a set of words and their definitions in which all the defined and defining words are defined.\nIf the Kernel size is too big, we might need to check a different dataset.")

We see that there are ,  27975  non defined words in our dictionnary
This is not consisent with the assumption that a Dictionary (D) is a set of words and their definitions in which all the defined and defining words are defined.
If the Kernel size is too big, we might need to check a different dataset.


In [13]:
print('Number of words in the kernel', str(count_kernel))
print("the past hypothesis was correct")

Number of words in the kernel 31329
the past hypothesis was correct


In [30]:
kernel

['german',
 'genus',
 'resembling',
 'alt',
 'first',
 'rod',
 'latin',
 'fifth',
 'also',
 'red-hot',
 'act',
 'abaculus',
 'abacus',
 'back',
 'radiate',
 'large',
 'abaculus',
 'glass',
 'abacus',
 'abacus',
 'table',
 'rhinoceros',
 'same',
 'from',
 'obeisance',
 'abashed',
 'from',
 'act',
 'univalve',
 'abandon',
 'abandon',
 'abandon',
 'reject',
 'legally',
 'act',
 'forfeited',
 'abnet',
 'palm',
 'alt',
 'free',
 'abase',
 'abase',
 'abase',
 'abjectly',
 'act',
 'he',
 'abash',
 'abash',
 'guilt',
 'abashed',
 'abashed',
 'alt',
 'about',
 'abated',
 'abate',
 'abate',
 'reduce',
 'act',
 'alt',
 'formed',
 'abatis',
 'nuisance',
 'abattoir',
 'for',
 'grass',
 'rostrum',
 'abashed',
 'alt',
 'from',
 'upon',
 'yarn',
 'father',
 'abbacy',
 'jurisdiction',
 'abbey',
 'abbatial',
 'french',
 'female',
 'abbey',
 'from',
 'head',
 'abbot',
 'abbreviate',
 'abbreviate',
 'abridge',
 'act',
 'reply',
 'abbreviate',
 'abbreviation',
 'abb',
 'first',
 'religious',
 'given',
 'wa

## Core extraction

In [14]:
g = Graph(n)
for key,value in dictionnary.items():
    for word in value[1]:
        if word in dictionnary:
            g.add_edge((id_mapping[key]),(id_mapping[word]))
core_sat = g.get_core_n_satellites(reverse_mapping)
core = core_sat[0]
print("The number of words in the core is ", str(len(core_sat[0])))
print("Those words are : ")
print(core)

The number of words in the core is  5618
Those words are : 
['german', 'accent', 'acute', 'acuteness', 'gravity', 'aluminium', 'alum', 'aluminous', 'hornblende', 'foliation', 'feathering', 'feather', 'calamus', 'rattan', 'umbilicus', 'navel', 'nave', 'aisle', 'screen', 'fight', 'abstraction', 'absence', 'calm', 'allay', 'allayed', 'unlaid', 'nettles', 'acalephae', 'acalephan', 'acalephs', 'jellyfish', 'umbrella', 'frame', 'abacus', 'aback', 'jam', 'jammed', 'jar', 'jarred', 'jug', 'large', 'abroad', 'air', 'absent', 'absent-minded', 'abstract', 'abstracted', 'remote', 'abstruse', 'recess', 'alcove', 'fireplace', 'gather', 'amass', 'accumulate', 'accumulated', 'accumulation', 'gain', 'access', 'accessible', 'accessibility', 'familiar', 'accustomed', 'forbearing', 'forbearance', 'abstinence', 'fast', 'adhere', 'adhered', 'routine', 'groove', 'flute', 'flageolet', 'recorder', 'register', 'calendar', 'almanac', 'calends', 'calendula', 'goolde', 'gold', 'abuse', 'abused', 'license', 'allow'

## Satellites exctraction

In [15]:
satellites = core_sat[1]
number = len(satellites)
print("The are is total ", str(number), " satellite sets")
for i in range(number):
    print("Set number ", str(i+1), " of size ", str(satellites[i][1]), str(satellites[i][0]))

The are is total  619  satellite sets
Set number  1  of size  2 ['zincic', 'zincous']
Set number  2  of size  2 ['zante', 'zantewood']
Set number  3  of size  2 ['yonker', 'younker']
Set number  4  of size  2 ['yeomanlike', 'yeomanly']
Set number  5  of size  2 ['xylene', 'xylol']
Set number  6  of size  2 ['xiphosura', 'xiphura']
Set number  7  of size  2 ['xiphiplastron', 'xiphisternum']
Set number  8  of size  2 ['xiphoid', 'xiphoidian']
Set number  9  of size  2 ['xanthorhiza', 'yellowroot']
Set number  10  of size  2 ['xanthelasma', 'xanthoma']
Set number  11  of size  2 ['water-ret', 'water-rot']
Set number  12  of size  2 ['wasite', 'wasium']
Set number  13  of size  2 ['washboard', 'wasteboard']
Set number  14  of size  2 ['yelp', 'yaup']
Set number  15  of size  2 ['vicinal', 'vicine']
Set number  16  of size  2 ['viameter', 'viatometer']
Set number  17  of size  2 ['vesiculose', 'vesiculous']
Set number  18  of size  2 ['vesicular', 'vesiculate']
Set number  19  of size  2 ['

Set number  225  of size  2 ['lac', 'lakh']
Set number  226  of size  2 ['labyrinthodon', 'labyrinthodonta']
Set number  227  of size  2 ['labyrinthal', 'labyrinthian']
Set number  228  of size  2 ['kidneywort', 'navelwort']
Set number  229  of size  3 ['kenogenesis', 'palingenesy', 'palingenesis']
Set number  230  of size  2 ['keeve', 'kier']
Set number  231  of size  3 ['regressive', 'retrogression', 'retrogressive']
Set number  232  of size  2 ['reticular', 'retiform']
Set number  233  of size  3 ['karyokinesis', 'nuclear', 'karyostenosis']
Set number  234  of size  2 ['kamsin', 'khamsin']
Set number  235  of size  3 ['kamichi', 'palamedeae', 'screamer']
Set number  236  of size  2 ['kaka', 'nestor']
Set number  237  of size  2 ['kail', 'kale']
Set number  238  of size  2 ['juglone', 'nucin']
Set number  239  of size  2 ['jougs', 'juggs']
Set number  240  of size  2 ['joe', 'johannes']
Set number  241  of size  4 ['veratrine', 'sabadilla', 'veratria', 'veratrina']
Set number  242  o

Set number  406  of size  2 ['gargle', 'gargling']
Set number  407  of size  5 ['warble', 'warbling', 'yodle', 'yodeling', 'yodel']
Set number  408  of size  2 ['rinse', 'rinsing']
Set number  409  of size  2 ['gantline', 'girtline']
Set number  410  of size  2 ['lashing', 'lasher']
Set number  411  of size  3 ['galloway', 'garran', 'garron']
Set number  412  of size  3 ['hotchpotch', 'hotchpot', 'hodgepodge']
Set number  413  of size  2 ['hash', 'hashed']
Set number  414  of size  3 ['galician', 'gallego', 'gallegan']
Set number  415  of size  2 ['galanga', 'galangal']
Set number  416  of size  2 ['galoche', 'galoshe']
Set number  417  of size  2 ['lamellar', 'lamelliform']
Set number  418  of size  2 ['gabardine', 'gaberdine']
Set number  419  of size  2 ['ligature', 'ligate']
Set number  420  of size  2 ['fundamentally', 'radically']
Set number  421  of size  2 ['fulminating', 'fulminate']
Set number  422  of size  2 ['fulahs', 'foolahs']
Set number  423  of size  2 ['fuchsine', 'ro

## Give the category of each word of a custom user written sentence

In [31]:
my_sentence = "the german admitted speaking hindi".lower() # please modify this sentence

In [32]:
for word in my_sentence.split(" "):
    tags = []
    if word in kernel:
        tags.append("Kernel")
    if word in core:
        tags.append("Core")
    for elm in satellites:
        if word in elm[0]:
            tags.append("Satellite")
            break
        
    if(tags==[]):
        print("The word ", word, " is neither in the kernel, the core or a satellite")
    else:
        print("The word ", word, " is in : ", " , ".join(tags))

The word  the  is neither in the kernel, the core or a satellite
The word  german  is in :  Kernel , Core
The word  admitted  is in :  Kernel , Core
The word  speaking  is neither in the kernel, the core or a satellite
The word  hindi  is in :  Satellite
